In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [3]:
# df0 is a mix of two datasets: "Collisions-Events" and "Collisions-Involved"
df0 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/involved2020_forinsight_r1.csv")
df0

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,26,29,30,31,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,collision_no,accyear,accdate,acctime,longitude,latitude,stname1,streetype1,dir1,stname2,streetype2,dir2,stname3,streetype3,dir3,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_no,vehicle_class,initial_dir,impact_type,involved_class,involved_injury_class,safety_equip_used,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,cyclist_action,cyclist_condition,cyclist_collision_type,manoeuver,posted_speed,actual_speed
0,1,2000,2000-09-11,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",NORTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING LEFT,NaN,NaN
1,1,2000,2000-09-11,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,2.0,"AUTOMOBILE, STATION WAGON",SOUTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,NaN,NaN
2,2,2000,2000-09-11,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,UNKNOWN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,2000,2000-09-11,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,NaN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,2000,2000-09-11,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,4.0,NaN,UNKNOWN,OTHER,VEHICLE OWNER,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING RIGHT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601707,1038396,2019,2019-05-10,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,3.0,"AUTOMOBILE, STATION WAGON",EAST,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,PARKED,40.0,0.0
2601708,1038396,2019,2019-05-10,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,2.0,NaN,NaN,TURNING MOVEMENT,PASSENGER,MINOR,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2601709,1038397,2019,2019-07-20,2236,-79.314602,43.678378,GERRARD ST E,NaN,NaN,BOWMORE RD,NaN,NaN,43 m East of,NaN,NaN,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",EAST,SMV UNATTENDED VEHICLE,DRIVER,NONE,USE UNKNOWN,OTHER,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,40.0,50.0
2601710,1038398,2019,2019-10-08,938,-79.372333,43.651519,JARVIS ST,NaN,NaN,ADELAIDE ST E,NaN,NaN,NaN,NaN,NaN,Major Arterial,CLEAR,DAYLIGHT,DRY,TRAFFIC SIGNAL,1.0,NaN,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,USE UNKNOWN,NaN,NaN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df0.columns

Index(['collision_no', 'accyear', 'accdate', 'acctime', 'longitude',
       'latitude', 'stname1', 'streetype1', 'dir1', 'stname2', 'streetype2',
       'dir2', 'stname3', 'streetype3', 'dir3', 'road_class', 'visibility',
       'light', 'road_surface_cond', 'traffic_control', 'vehicle_no',
       'vehicle_class', 'initial_dir', 'impact_type', 'involved_class',
       'involved_injury_class', 'safety_equip_used', 'driver_action',
       'driver_condition', 'pedestrian_action', 'pedestrian_condition',
       'pedestrian_collision_type', 'cyclist_action', 'cyclist_condition',
       'cyclist_collision_type', 'manoeuver', 'posted_speed', 'actual_speed'],
      dtype='object')

In [5]:
# This shows the number of collisions itself (NOT the number of people involved in the collisions)
df0['collision_no'].nunique()

1038398

---
##df1
---

In [0]:
# Create df1 from df0
df1 = df0.copy()

# Add new column 'acc_date'
df1['acc_date'] = pd.to_datetime(df0['accdate'], errors = 'raise')
df1['year'] = df1['acc_date'].dt.year
df1['month_name'] = df1['acc_date'].dt.month_name()
df1['day_name'] = df1['acc_date'].dt.day_name()

# Drop columns 'accdate' and 'accyear'
df1.drop(['accdate', 'accyear'], axis=1, inplace=True)

In [7]:
df1['acc_date'].dt.year.unique() 

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [8]:
df1['involved_injury_class'].unique()

array(['NONE', 'MINIMAL', nan, 'MINOR', 'FATAL', 'MAJOR'], dtype=object)

In [9]:
print('{:,}'.format(df1['involved_injury_class'].isna().sum()), 
      'data points in "involved_injury_class" are nan, which will be removed.')
df1.dropna(subset=['involved_injury_class'], inplace=True)

137,349 data points in "involved_injury_class" are nan, which will be removed.


In [10]:
df1['involved_class'].unique()

array(['DRIVER', 'PASSENGER', 'VEHICLE OWNER', nan, 'OTHER',
       'OTHER PROPERTY OWNER', 'TRUCK DRIVER', 'PEDESTRIAN', 'CYCLIST',
       'WITNESS', 'MOTORCYCLE DRIVER', 'RUNAWAY - NO DRIVER',
       'WHEELCHAIR', 'IN-LINE SKATER', 'MOPED DRIVER', 'UNKNOWN - FTR',
       'DRIVER - NOT HIT', 'TRAILER OWNER', 'MOTORCYCLE PASSENGER',
       'CYCLIST - NOT HIT', 'MOPED PASSENGER', 'CYCLIST PASSENGER',
       'ANIMAL', 'PEDESTRIAN - NOT HIT'], dtype=object)

In [11]:
print('{:,}'.format(df1['involved_class'].isna().sum()), 
      'data points in "involved_class" are nan, which will be removed.')
df1.dropna(subset=['involved_class'], inplace=True)

12,228 data points in "involved_class" are nan, which will be removed.


In [12]:
print('{:,}'.format(df1['longitude'].isna().sum()), 
      'data points in "longitude" are nan, which will be removed.')
df1.dropna(subset=['longitude'], inplace=True)

21 data points in "longitude" are nan, which will be removed.


In [13]:
df1['latitude'].isna().sum()

0

In [14]:
# Drop points whose lon-lat are not in Toronto (spatial outlier)
# Toronto lon-lat ranges:
#    longitude: (-79.65 , -79.08)
#    latitude: (43.57 , 43.85)

i = df1[(df1['longitude']<-79.65) | (df1['longitude']>-79.08) | (df1['latitude']<43.57) | (df1['latitude']>43.85)].index
df1.drop(i, inplace=True)

print('There were {:,} spatial outliers which are removed now.'.format(i.size))

There were 2,421 spatial outliers which are removed now.


In [15]:
# Add new columns to df1
df1['# involved'] = int(1)

df1['# injured'] = int(0)
df1.loc[df1['involved_injury_class'].isin(['MINIMAL', 'MINOR', 'MAJOR']), ['# injured']] = int(1) 

df1['# fatalities'] = int(0)
df1.loc[df1['involved_injury_class']=='FATAL', ['# fatalities']] = int(1) 

df1['# KSI'] = int(0)
df1.loc[df1['involved_injury_class'].isin(['MAJOR', 'FATAL']), ['# KSI']] = int(1) 
df1

,collision_no,acctime,longitude,latitude,stname1,streetype1,dir1,stname2,streetype2,dir2,stname3,streetype3,dir3,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_no,vehicle_class,initial_dir,impact_type,involved_class,involved_injury_class,safety_equip_used,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,cyclist_action,cyclist_condition,cyclist_collision_type,manoeuver,posted_speed,actual_speed,acc_date,year,month_name,day_name,# involved,# injured,# fatalities,# KSI
0,1,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",NORTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING LEFT,NaN,NaN,2000-09-11,2000,September,Monday,1,0,0,0
1,1,850,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,2.0,"AUTOMOBILE, STATION WAGON",SOUTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,NaN,NaN,2000-09-11,2000,September,Monday,1,0,0,0
2,2,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,UNKNOWN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-09-11,2000,September,Monday,1,1,0,0
3,2,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,NaN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-09-11,2000,September,Monday,1,1,0,0
4,2,850,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,4.0,NaN,UNKNOWN,OTHER,VEHICLE OWNER,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING RIGHT,NaN,NaN,2000-09-11,2000,September,Monday,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601707,1038396,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,3.0,"AUTOMOBILE, STATION WAGON",EAST,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,PARKED,40.0,0.0,2019-05-10,2019,May,Friday,1,0,0,0
2601708,1038396,2200,-79.351940,43.677554,DANFORTH AV,NaN,NaN,CHESTER AV,NaN,NaN,22 m East of,NaN,NaN,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,2.0,NaN,NaN,TURNING MOVEMENT,PASSENGER,MINOR,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-10,2019,May,Friday,1,1,0,0
2601709,1038397,2236,-79.314602,43.678378,GERRARD ST E,NaN,NaN,BOWMORE RD,NaN,NaN,43 m East of,NaN,NaN,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",EAST,SMV UNATTENDED VEHICLE,DRIVER,NONE,USE UNKNOWN,OTHER,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,40.0,50.0,2019-07-20,2019,July,Saturday,1,0,0,0
2601710,1038398,938,-79.372333,43.651519,JARVIS ST,NaN,NaN,ADELAIDE ST E,NaN,NaN,NaN,NaN,NaN,Major Arterial,CLEAR,DAYLIGHT,DRY,TRAFFIC SIGNAL,1.0,NaN,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,USE UNKNOWN,NaN,NaN,NaN,NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-08,2019,October,Tuesday,1,1,0,0


In [0]:
############### In the 'Main' notebook, here df1E is created: the dataframe of "Collisions-Events" from df1 ###################################################################

---
##df2
---

In [0]:
# Create df2 from df1 and reset its index (as some rows are already dropped)
df2 = df1.copy()
df2.reset_index(inplace=True)

In [0]:
# Drop 'month_name' and 'day_name' as they are string, we're going to extract the integer values from 'acc_date'
df2.drop(['year', 'month_name', 'day_name'],axis=1, inplace=True)

In [19]:
# extract hour from acctime (integer)

hour = []
for i in range(len(df2)):
    if len(str(df2.loc[i, 'acctime'])) == 2:
        hour.append(0)
    elif len(str(df2.loc[i, 'acctime'])) == 3:
        hour.append(int(str(df2.loc[i, 'acctime'])[:1]))
    elif len(str(df2.loc[i, 'acctime'])) == 4:
        hour.append(int(str(df2.loc[i, 'acctime'])[:2]))
    else:
        hour.append(np.nan)
        
df2['hour'] = hour

df2.drop('acctime', axis=1, inplace=True)

df2['hour'].unique()

array([ 8.,  6., 11., 17., 13., 14., 16., 19.,  2., 18., 20., 22., 15.,
       21.,  1.,  7., 23.,  9., 12., 10.,  0., nan,  3.,  4.,  5.])

In [20]:
print('# nan in "hour": {:,}'.format(df2['hour'].isna().sum()))

# nan in "hour": 11,945


In [0]:
# Drop nan values of hour
df2.dropna(subset=['hour'], inplace=True )
df2['hour'] = df2['hour'].astype(np.int8)

In [0]:
df2['year'] = df2['acc_date'].dt.year
df2['month'] = df2['acc_date'].dt.month
df2['day'] = df2['acc_date'].dt.dayofweek

In [23]:
df2['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [24]:
df2['month'].unique()

array([ 9,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12])

In [25]:
df2['day'].unique()

array([0, 6, 1, 2, 3, 4, 5])

In [26]:
df2.drop('index', axis=1, inplace=True)
df2.head()

,collision_no,longitude,latitude,stname1,streetype1,dir1,stname2,streetype2,dir2,stname3,streetype3,dir3,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_no,vehicle_class,initial_dir,impact_type,involved_class,involved_injury_class,safety_equip_used,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,cyclist_action,cyclist_condition,cyclist_collision_type,manoeuver,posted_speed,actual_speed,acc_date,# involved,# injured,# fatalities,# KSI,hour,year,month,day
0,1,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,1.0,"AUTOMOBILE, STATION WAGON",NORTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING LEFT,NaN,NaN,2000-09-11,1,0,0,0,8,2000,9,0
1,1,-79.456317,43.727732,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,2.0,"AUTOMOBILE, STATION WAGON",SOUTH,TURNING MOVEMENT,DRIVER,NONE,USE UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,NaN,NaN,2000-09-11,1,0,0,0,8,2000,9,0
2,2,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,UNKNOWN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-09-11,1,1,0,0,8,2000,9,0
3,2,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,0.0,NaN,NaN,OTHER,PASSENGER,MINIMAL,LAP AND SHOULDER BELT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-09-11,1,1,0,0,8,2000,9,0
4,2,-79.456271,43.727698,YORKDALE,RD,W,401 C E YORKDALE W RAMP,RD,NaN,NaN,NaN,NaN,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,4.0,NaN,UNKNOWN,OTHER,VEHICLE OWNER,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING RIGHT,NaN,NaN,2000-09-11,1,0,0,0,8,2000,9,0


In [27]:
for column in df2.columns:
    print('# nan in "{}": {:,}'.format(column, df2[column].isna().sum()))

# nan in "collision_no": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "stname1": 5
# nan in "streetype1": 608,509
# nan in "dir1": 1,804,201
# nan in "stname2": 326,712
# nan in "streetype2": 843,397
# nan in "dir2": 2,095,412
# nan in "stname3": 2,332,057
# nan in "streetype3": 2,437,748
# nan in "dir3": 2,437,748
# nan in "road_class": 94,989
# nan in "visibility": 6,822
# nan in "light": 249
# nan in "road_surface_cond": 1,334
# nan in "traffic_control": 13,036
# nan in "vehicle_no": 48
# nan in "vehicle_class": 160,701
# nan in "initial_dir": 557,695
# nan in "impact_type": 279
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "safety_equip_used": 595,191
# nan in "driver_action": 1,282,534
# nan in "driver_condition": 1,289,126
# nan in "pedestrian_action": 2,401,248
# nan in "pedestrian_condition": 2,399,988
# nan in "pedestrian_collision_type": 2,405,553
# nan in "cyclist_action": 2,422,038
# nan in "cyclist_condition": 2,437,748
# nan in "cy

---
###df2_b
Here before removing some of the columns which have many *nan* or *UNKNOWN* values, I create a branch of df2 to do analysis on the most important features that cause KSI accidents for **pedestrians**, **drivers**, and **cyclists**.

---

In [0]:
df2_b = df2.copy()

In [29]:
df2_b['involved_class'].value_counts()

DRIVER                  1799730
VEHICLE OWNER            257843
PASSENGER                171307
OTHER                     72867
TRUCK DRIVER              45321
PEDESTRIAN                38223
CYCLIST                   22693
OTHER PROPERTY OWNER      21078
MOTORCYCLE DRIVER          6777
DRIVER - NOT HIT            397
MOPED DRIVER                364
MOTORCYCLE PASSENGER        277
WHEELCHAIR                  245
UNKNOWN - FTR               148
RUNAWAY - NO DRIVER         140
WITNESS                     127
IN-LINE SKATER               70
CYCLIST PASSENGER            68
PEDESTRIAN - NOT HIT         27
MOPED PASSENGER              14
ANIMAL                       13
TRAILER OWNER                11
CYCLIST - NOT HIT             8
Name: involved_class, dtype: int64

In [30]:
df2_b['streetype1'].unique()

array(['RD', 'ST', 'Stre', 'Road', 'Aven', 'BLVD', 'AVE', 'Driv', nan,
       'Cres', 'DR', 'Boul', 'Cour', 'Expr', 'LANE', 'Gate', 'RAMP',
       'Trai', 'Heig', 'Way', 'Circ', 'Plac', 'Ramp', 'Line', 'CRCL',
       'Squa', 'CRES', 'Park', 'PL', 'Ave', 'Rd', 'TRL', 'GDNS', 'Grov',
       'CRT', 'Terr', 'QUAY', 'Gard', 'GT', 'WAY', 'PK', 'St', 'PKWY',
       'TER', 'SQ', 'Quay', 'Gree', 'Blvd', 'Path', 'dr', 'Brid', 'AVE ',
       'Dr', 'Lane', 'HTS', 'EXPY', 'Ridg', 'Clos', 'GRV', 'Sq', 'ROAD',
       'CR', 'Orch', 'MEWS', 'WALK', 'Bluf', 'st', 'Str', 'PTWY', 'AvE',
       'EXP', 'E', 'Trl', 'XY', 'Elmw', 'Airp', 'ST ', 'Grv', 'LINE',
       'Crt', 'Pl', 'ave', 'High', 'C', 'Crct', 'W', 'Lawn', 'rd', 'TrL',
       'Foxw', 'blvd', 'Gt', 'Tr', 'Bend', 'CRCT', 'CIR', 'Crcl', 'Ave ',
       'ramp', 'VE', 'Acre', 'cres', 'Rod', 'Exp', 'Ter', 'AVe', 'DD',
       'PATH', 'crt', 'AvVE', 'ABE', 'crcl', 'CT', 'w', 'N', 'ExP',
       'Mews', 'S', 'PARK', 'Dri', 'MALL', 'BLD ', 'Pkwy', 'SY', 'BlV

In [31]:
df2_b['safety_equip_used'].unique()

array(['USE UNKNOWN', 'LAP AND SHOULDER BELT', nan, 'LAP BELT ONLY',
       'NO EQUIPMENT AVAILABLE', 'LAP BELT ONLY OF COMBINED ASSEMB',
       'OTHER SAFETY EQUIPMENT USED', 'EQUIPMENT NOT USED BUT AVAILABLE',
       'AIR BAG DEPLOYED', 'HELMET', 'CHILD SAFETY SEAT USED CORRECTLY',
       'OTHER PASSIVE RESTRAINT DEVICE',
       'CHILD SAFETY SEAT USED INCORRECTLY'], dtype=object)

In [32]:
df2_b['manoeuver'].unique()

array(['TURNING LEFT', 'GOING AHEAD', nan, 'TURNING RIGHT',
       'SLOWING OR STOPPING', 'PULLING AWAY FROM SHOULDER OR CURB',
       'UNKNOWN', 'STOPPED', 'CHANGING LANES', 'PARKED',
       'PULLING ONTO SHOULDER OR TOWARDCURB', 'REVERSING', 'OTHER',
       'MAKING "U" TURN', 'DISABLED', 'OVERTAKING', 'MERGING',
       'STOPPED OR PARKED'], dtype=object)

In [33]:
print('In addition to {:,} nan values for "manoeuver", there are {:,} "UNKNOWN" (not "OTHER") elements in it.'
    .format(df2_b['manoeuver'].isna().sum(), 
            df2[df2_b['manoeuver']=='UNKNOWN'].shape[0]))

In addition to 259,040 nan values for "manoeuver", there are 322,782 "UNKNOWN" (not "OTHER") elements in it.


In [34]:
# Drop the following columns form df2_b as they contain too many 'nan' and also UNKNOWN elements or they are not useful for the analysis. 
df2_b.drop(['stname1', 'streetype1', 'dir1', 'stname2', 'streetype2', 'dir2', 'stname3', 'streetype3', 'dir3', 'acc_date',
            'initial_dir','safety_equip_used', 'vehicle_no', 'involved_injury_class', '# involved',	'# injured',	'# fatalities',
            'posted_speed', 'actual_speed'], axis=1, inplace=True)
df2_b.head()

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_class,impact_type,involved_class,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,cyclist_action,cyclist_condition,cyclist_collision_type,manoeuver,# KSI,hour,year,month,day
0,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING LEFT,0,8,2000,9,0
1,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,0,8,2000,9,0
2,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,NaN,OTHER,PASSENGER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,8,2000,9,0
3,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,NaN,OTHER,PASSENGER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,8,2000,9,0
4,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,NaN,OTHER,VEHICLE OWNER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURNING RIGHT,0,8,2000,9,0


---
#### **Pedestrian**
---

In [35]:
# Create df2_b_p
df2_b_p = df2_b.copy()
df2_b_p = df2_b_p[df2_b_p['involved_class']=='PEDESTRIAN']
df2_b_p.drop(['cyclist_action',	'cyclist_condition',	'cyclist_collision_type'], axis=1, inplace=True)
df2_b_p

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_class,impact_type,involved_class,driver_action,driver_condition,pedestrian_action,pedestrian_condition,pedestrian_collision_type,manoeuver,# KSI,hour,year,month,day
806,368,-79.413448,43.761828,Minor Arterial,CLEAR,DAYLIGHT,DRY,STOP SIGN,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,NaN,NaN,NaN,GOING AHEAD,1,13,2000,1,3
931,435,-79.491039,43.763491,Major Arterial,CLEAR,DAYLIGHT,DRY,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,CROSSING WITHOUT RIGHT OF WAY,INATTENTIVE,NaN,NaN,0,8,2000,1,2
939,439,-79.545238,43.601679,Major Arterial,CLEAR,DARK,WET,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,OTHER,"ABILITY IMPAIRED, ALCOHOL",NaN,NaN,1,23,2000,1,6
2573,1170,-79.323344,43.684256,Local,SNOW,"DAYLIGHT, ARTIFICIAL",LOOSE SNOW,NO CONTROL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,ON SIDEWALK OR SHOULDER,NORMAL,NaN,NaN,0,7,2000,1,3
2635,1200,-79.337573,43.668784,Minor Arterial,CLEAR,DAYLIGHT,SLUSH,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,CROSSING WITH RIGHT OF WAY,NaN,NaN,TURNING LEFT,0,11,2000,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449659,1038383,-79.388943,43.656348,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,NaN,NORMAL,NaN,NaN,0,5,2019,12,0
2449662,1038384,-79.377338,43.647845,Major Arterial,CLEAR,DAYLIGHT,WET,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,NaN,NORMAL,NaN,NaN,0,8,2019,12,0
2449665,1038385,-79.397976,43.639392,Collector,CLEAR,"DARK, ARTIFICIAL",DRY,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,NaN,NORMAL,NaN,NaN,0,18,2019,12,0
2449669,1038388,-79.388027,43.766586,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,NaN,PEDESTRIAN COLLISIONS,PEDESTRIAN,NaN,NaN,NaN,NORMAL,NaN,NaN,0,6,2019,12,1


In [36]:
df2_b_p['pedestrian_action'].unique()

array([nan, 'CROSSING WITHOUT RIGHT OF WAY', 'OTHER',
       'ON SIDEWALK OR SHOULDER', 'CROSSING WITH RIGHT OF WAY',
       'COMING FROM BEHIND PARKED VEHICLE',
       'CROSSING, NO TRAFFIC CONTROL', 'RUNNING ONTO ROADWAY',
       'WALKING ON ROADWAY WITH TRAFFIC', 'PERSON GETTING ON/OFF VEHICLE',
       'CROSSING, PEDESTRIAN CROSSOVER', 'PUSHING/WORKING ON VEHICLE',
       'PLAYING OR WORKING ON HIGHWAY',
       'CROSSING MARKED CROSSWALK WITHOUT ROW',
       'WALKING ON ROADWAY AGAINST TRAFFIC',
       'PERSON GETTING ON/OFF SCHOOL BUS'], dtype=object)

In [37]:
print('Out of {:,} "PEDESTRIAN" data points, {:,} "pedestrian_action" is nan, which are going to be dropped.'.
      format(df2_b_p.shape[0], df2_b_p[df2_b_p['pedestrian_action'].isna()].shape[0]))

Out of 38,223 "PEDESTRIAN" data points, 2,154 "pedestrian_action" is nan, which are going to be dropped.


In [0]:
i = df2_b_p[df2_b_p['pedestrian_action'].isna()].index
df2_b_p.drop(i, inplace=True)

In [39]:
df2_b_p['pedestrian_condition'].unique()

array(['INATTENTIVE', 'ABILITY IMPAIRED, ALCOHOL', 'NORMAL', nan,
       'UNKNOWN', 'HAD BEEN DRINKING', 'MEDICAL OR PHYSICAL DISABILITY',
       'OTHER', 'ABILITY IMPAIRED, DRUGS',
       'ABILITY IMPAIRED, ALCOHOL OVER .80', 'FATIGUE'], dtype=object)

In [40]:
print('Out of {:,} "PEDESTRIAN" data points, {:,} "pedestrian_condition" is nan, which are going to be dropped.'.
      format(df2_b_p.shape[0], df2_b_p[df2_b_p['pedestrian_condition'].isna()].shape[0]))

Out of 36,069 "PEDESTRIAN" data points, 251 "pedestrian_condition" is nan, which are going to be dropped.


In [0]:
i = df2_b_p[df2_b_p['pedestrian_condition'].isna()].index
df2_b_p.drop(i, inplace=True)

In [42]:
df2_b_p['pedestrian_collision_type'].unique()

array([nan, 'PEDESTRIAN HIT AT PRIVATE DRIVEWAY',
       'PEDESTRIAN HIT ON SIDEWALK OR SHOULDER',
       'VEHICLE TURNS LEFT WHILE PED CROSSES WITH ROW AT INTER.',
       'PEDESTRIAN HIT AT MID-BLOCK', 'OTHER / UNDEFINED',
       'VEHICLE IS GOING STRAIGHT THRU INTER.WHILE PED CROSS WITH ROW',
       'VEHICLE TURNS RIGHT WHILE PED CROSSES WITH ROW AT INTER.',
       'VEHICLE IS REVERSING AND HITS PEDESTRIAN',
       'VEHICLE HITS THE PEDESTRIAN WALKING OR RUNNING OUT FROM BETWEEN PARKED VEHICLES AT MID-BLOCK',
       'PEDESTRIAN HIT A PXO/PED. MID-BLOCK SIGNAL',
       'VEHICLE IS GOING STRAIGHT THRU INTER.WHILE PED CROSS WITHOUT ROW',
       'PEDESTRIAN INVOLVED IN A COLLISION WITH TRANSIT VEHICLE ANYWHERE ALONG ROADWAY',
       'VEHICLE TURNS LEFT WHILE PED CROSSES WITHOUT ROW AT INTER.',
       'UNKNOWN',
       'VEHICLE TURNS RIGHT WHILE PED CROSSES WITHOUT ROW AT INTER.',
       'PEDESTRIAN HIT AT PARKING LOT'], dtype=object)

In [43]:
print('Out of {:,} "PEDESTRIAN" data points, {:,} "pedestrian_collision_type" is nan, which are going to be dropped.'.
      format(df2_b_p.shape[0], df2_b_p[df2_b_p['pedestrian_collision_type'].isna()].shape[0]))

Out of 35,818 "PEDESTRIAN" data points, 4,237 "pedestrian_collision_type" is nan, which are going to be dropped.


In [0]:
i = df2_b_p[df2_b_p['pedestrian_collision_type'].isna()].index
df2_b_p.drop(i, inplace=True)

In [45]:
for column in df2_b_p.columns:
    print('# nan in "{}": {:,}'.format(column, df2_b_p[column].isna().sum()))

# nan in "collision_no": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "road_class": 62
# nan in "visibility": 17
# nan in "light": 0
# nan in "road_surface_cond": 15
# nan in "traffic_control": 4
# nan in "vehicle_class": 9,277
# nan in "impact_type": 0
# nan in "involved_class": 0
# nan in "driver_action": 31,581
# nan in "driver_condition": 31,581
# nan in "pedestrian_action": 0
# nan in "pedestrian_condition": 0
# nan in "pedestrian_collision_type": 0
# nan in "manoeuver": 30,791
# nan in "# KSI": 0
# nan in "hour": 0
# nan in "year": 0
# nan in "month": 0
# nan in "day": 0


In [46]:
# Drop the following columns: "driver_action", "driver_condition", "manoeuver"
df2_b_p.drop(["driver_action", "driver_condition", "manoeuver"], axis=1, inplace=True)

for column in df2_b_p.columns:
  if df2_b_p[column].isna().sum()!=0:
    print('# nan in "{}": {:,}'.format(column, df2_b_p[column].isna().sum()))

# nan in "road_class": 62
# nan in "visibility": 17
# nan in "road_surface_cond": 15
# nan in "traffic_control": 4
# nan in "vehicle_class": 9,277


In [47]:
df2_b_p.dropna(subset=["road_class", "visibility", "road_surface_cond", "traffic_control", "vehicle_class"], inplace=True)

df2_b_p.isna().sum()

collision_no                 0
longitude                    0
latitude                     0
road_class                   0
visibility                   0
light                        0
road_surface_cond            0
traffic_control              0
vehicle_class                0
impact_type                  0
involved_class               0
pedestrian_action            0
pedestrian_condition         0
pedestrian_collision_type    0
# KSI                        0
hour                         0
year                         0
month                        0
day                          0
dtype: int64

In [48]:
df2_b_p

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_class,impact_type,involved_class,pedestrian_action,pedestrian_condition,pedestrian_collision_type,# KSI,hour,year,month,day
113810,46149,-79.377467,43.646487,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,BICYCLE,PEDESTRIAN COLLISIONS,PEDESTRIAN,ON SIDEWALK OR SHOULDER,NORMAL,PEDESTRIAN HIT ON SIDEWALK OR SHOULDER,0,11,2000,8,1
169549,68554,-79.371021,43.658282,Major Arterial,CLEAR,"DARK, ARTIFICIAL",WET,TRAFFIC SIGNAL,OTHER,PEDESTRIAN COLLISIONS,PEDESTRIAN,CROSSING WITH RIGHT OF WAY,NORMAL,VEHICLE TURNS LEFT WHILE PED CROSSES WITH ROW ...,0,1,2000,12,6
185194,74580,-79.419811,43.797145,Major Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,BICYCLE,PEDESTRIAN COLLISIONS,PEDESTRIAN,RUNNING ONTO ROADWAY,NORMAL,PEDESTRIAN HIT AT MID-BLOCK,0,14,2001,1,0
334212,130958,-79.400290,43.652545,Major Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,OTHER,PEDESTRIAN COLLISIONS,PEDESTRIAN,CROSSING WITHOUT RIGHT OF WAY,UNKNOWN,PEDESTRIAN HIT AT MID-BLOCK,0,8,2002,1,0
334495,131080,-79.500590,43.722545,Major Arterial,CLEAR,DAYLIGHT,WET,NO CONTROL,OTHER,PEDESTRIAN COLLISIONS,PEDESTRIAN,"CROSSING, NO TRAFFIC CONTROL",NORMAL,PEDESTRIAN HIT AT MID-BLOCK,1,18,2002,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359937,987546,-79.430232,43.776503,Major Arterial,RAIN,"DARK, ARTIFICIAL",WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",PEDESTRIAN COLLISIONS,PEDESTRIAN,CROSSING WITH RIGHT OF WAY,INATTENTIVE,VEHICLE TURNS LEFT WHILE PED CROSSES WITH ROW ...,0,22,2018,12,3
2368690,991748,-79.510494,43.624591,Major Arterial,RAIN,"DAYLIGHT, ARTIFICIAL",WET,NO CONTROL,OTHER,PEDESTRIAN COLLISIONS,PEDESTRIAN,CROSSING WITH RIGHT OF WAY,NORMAL,VEHICLE TURNS LEFT WHILE PED CROSSES WITH ROW ...,0,16,2018,5,3
2373822,993828,-79.376651,43.649287,Major Arterial,CLEAR,DAYLIGHT,DRY,STOP SIGN,"AUTOMOBILE, STATION WAGON",PEDESTRIAN COLLISIONS,PEDESTRIAN,CROSSING WITH RIGHT OF WAY,NORMAL,VEHICLE IS GOING STRAIGHT THRU INTER.WHILE PED...,0,10,2018,7,1
2377602,995310,-79.504077,43.600909,Major Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,OTHER,PEDESTRIAN COLLISIONS,PEDESTRIAN,"CROSSING, NO TRAFFIC CONTROL",INATTENTIVE,VEHICLE IS GOING STRAIGHT THRU INTER.WHILE PED...,1,17,2018,9,1


In [0]:
df2_b_p.to_csv('df2_b_p.csv')

---
#### **Driver**
---

In [50]:
# Create df2_b_p
df2_b_d = df2_b.copy()
df2_b_d = df2_b_d[df2_b_d['involved_class']=='DRIVER']
df2_b_d.drop(['pedestrian_action',	'pedestrian_condition',	'pedestrian_collision_type', 
              'cyclist_action',	'cyclist_condition',	'cyclist_collision_type'], axis=1, inplace=True)
df2_b_d

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_class,impact_type,involved_class,driver_action,driver_condition,manoeuver,# KSI,hour,year,month,day
0,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,NaN,NaN,TURNING LEFT,0,8,2000,9,0
1,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,NaN,NaN,GOING AHEAD,0,8,2000,9,0
6,3,-79.398357,43.706721,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",SIDESWIPE,DRIVER,DRIVING PROPERLY,NORMAL,SLOWING OR STOPPING,0,6,2000,1,6
8,3,-79.398357,43.706721,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",SIDESWIPE,DRIVER,DRIVING PROPERLY,NORMAL,GOING AHEAD,0,6,2000,1,6
9,4,-79.395091,43.691344,Major Arterial,RAIN,DAYLIGHT,WET,NO CONTROL,"AUTOMOBILE, STATION WAGON",ANGLE,DRIVER,NaN,NaN,PULLING AWAY FROM SHOULDER OR CURB,0,11,2000,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449686,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,LOST CONTROL,MEDICAL OR PHYSICAL DISABILITY,GOING AHEAD,1,22,2019,5,4
2449687,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,DRIVING PROPERLY,NORMAL,"MAKING ""U"" TURN",0,22,2019,5,4
2449688,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,NaN,NORMAL,PARKED,0,22,2019,5,4
2449690,1038397,-79.314602,43.678378,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,NO CONTROL,"AUTOMOBILE, STATION WAGON",SMV UNATTENDED VEHICLE,DRIVER,OTHER,UNKNOWN,GOING AHEAD,0,22,2019,7,5


In [51]:
df2['driver_action'].unique()

array([nan, 'DRIVING PROPERLY', 'FAILED TO YIELD RIGHT OF WAY',
       'IMPROPER LANE CHANGE', 'IMPROPER TURN',
       'DISOBEYED TRAFFIC CONTROL', 'FOLLOWING TOO CLOSE', 'OTHER',
       'LOST CONTROL', 'SPEED TOO FAST FOR CONDITION', 'IMPROPER PASSING',
       'EXCEEDING SPEED LIMIT', 'SPEED TOO SLOW',
       'WRONG WAY ON ONE WAY ROAD'], dtype=object)

In [52]:
print('Out of {:,} "DRIVER" data points, {:,} "driver_action" is nan, which are going to be dropped.'.
      format(df2_b_d.shape[0], df2_b_d[df2_b_d['driver_action'].isna()].shape[0]))

Out of 1,799,730 "DRIVER" data points, 686,770 "driver_action" is nan, which are going to be dropped.


In [0]:
i = df2_b_d[df2_b_d['driver_action'].isna()].index
df2_b_d.drop(i, inplace=True)

In [54]:
for column in df2_b_d.columns:
    print('# nan in "{}": {:,}'.format(column, df2_b_d[column].isna().sum()))

# nan in "collision_no": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "road_class": 55,410
# nan in "visibility": 4,121
# nan in "light": 57
# nan in "road_surface_cond": 111
# nan in "traffic_control": 2,001
# nan in "vehicle_class": 4,137
# nan in "impact_type": 52
# nan in "involved_class": 0
# nan in "driver_action": 0
# nan in "driver_condition": 21,475
# nan in "manoeuver": 155
# nan in "# KSI": 0
# nan in "hour": 0
# nan in "year": 0
# nan in "month": 0
# nan in "day": 0


In [55]:
df2_b_d.dropna(subset=["road_class", "visibility", "light", "road_surface_cond", "traffic_control", 
                       "vehicle_class", "impact_type", "driver_condition", "manoeuver"], inplace=True)

df2_b_d.isna().sum()

collision_no         0
longitude            0
latitude             0
road_class           0
visibility           0
light                0
road_surface_cond    0
traffic_control      0
vehicle_class        0
impact_type          0
involved_class       0
driver_action        0
driver_condition     0
manoeuver            0
# KSI                0
hour                 0
year                 0
month                0
day                  0
dtype: int64

In [56]:
df2_b_d

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,traffic_control,vehicle_class,impact_type,involved_class,driver_action,driver_condition,manoeuver,# KSI,hour,year,month,day
6,3,-79.398357,43.706721,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",SIDESWIPE,DRIVER,DRIVING PROPERLY,NORMAL,SLOWING OR STOPPING,0,6,2000,1,6
8,3,-79.398357,43.706721,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",SIDESWIPE,DRIVER,DRIVING PROPERLY,NORMAL,GOING AHEAD,0,6,2000,1,6
32,13,-79.268306,43.787566,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,DRIVING PROPERLY,NORMAL,GOING AHEAD,0,17,2000,1,6
33,13,-79.268306,43.787566,Major Arterial,RAIN,DARK,WET,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,FAILED TO YIELD RIGHT OF WAY,NORMAL,TURNING LEFT,0,17,2000,1,6
65,28,-79.545291,43.601744,Major Arterial,CLEAR,DAYLIGHT,DRY,TRAFFIC SIGNAL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,DRIVING PROPERLY,NORMAL,GOING AHEAD,0,17,2000,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449685,1038395,-79.587139,43.668288,Local,CLEAR,DAYLIGHT,DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",SIDESWIPE,DRIVER,OTHER,UNKNOWN,GOING AHEAD,0,14,2019,10,6
2449686,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,LOST CONTROL,MEDICAL OR PHYSICAL DISABILITY,GOING AHEAD,1,22,2019,5,4
2449687,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,NO CONTROL,"AUTOMOBILE, STATION WAGON",TURNING MOVEMENT,DRIVER,DRIVING PROPERLY,NORMAL,"MAKING ""U"" TURN",0,22,2019,5,4
2449690,1038397,-79.314602,43.678378,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,NO CONTROL,"AUTOMOBILE, STATION WAGON",SMV UNATTENDED VEHICLE,DRIVER,OTHER,UNKNOWN,GOING AHEAD,0,22,2019,7,5


In [0]:
df2_b_d.to_csv('df2_b_d.csv')

---
### Rest of df2
---

In [0]:
# Drop the following columns form df2 as they contain too many 'nan' and also UNKNOWN elements. 
df2.drop(['streetype1', 'dir1', 'streetype2', 'dir2', 'streetype3', 'dir3', 'initial_dir', 'safety_equip_used', 
          'driver_action', 'driver_condition', 'pedestrian_action', 'pedestrian_condition', 'pedestrian_collision_type', 'cyclist_action', 'cyclist_condition', 'cyclist_collision_type',
         'manoeuver', 'posted_speed', 'actual_speed'], axis=1, inplace=True)

In [59]:
for column in df2.columns:
    print('# nan in "{}": {:,}'.format(column, df2[column].isna().sum()))

# nan in "collision_no": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "stname1": 5
# nan in "stname2": 326,712
# nan in "stname3": 2,332,057
# nan in "road_class": 94,989
# nan in "visibility": 6,822
# nan in "light": 249
# nan in "road_surface_cond": 1,334
# nan in "traffic_control": 13,036
# nan in "vehicle_no": 48
# nan in "vehicle_class": 160,701
# nan in "impact_type": 279
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "acc_date": 0
# nan in "# involved": 0
# nan in "# injured": 0
# nan in "# fatalities": 0
# nan in "# KSI": 0
# nan in "hour": 0
# nan in "year": 0
# nan in "month": 0
# nan in "day": 0


In [60]:
df2['impact_type'].unique()

array(['TURNING MOVEMENT', 'OTHER', 'SIDESWIPE', 'ANGLE', 'REAR END',
       'SMV UNATTENDED VEHICLE', 'APPROACHING', 'SMV OTHER',
       'PEDESTRIAN COLLISIONS', 'CYCLIST COLLISIONS', nan], dtype=object)

In [61]:
df2['vehicle_class'].unique()

array(['AUTOMOBILE, STATION WAGON', nan, 'TAXI', 'TRUCK - OPEN',
       'MUNICIPAL TRANSIT BUS (TTC)', 'OTHER', 'BICYCLE', 'RICKSHAW',
       'MOTORCYCLE', 'TRUCK - CLOSED (BLAZER, ETC)', 'PASSENGER VAN',
       'MOPED', 'DELIVERY VAN', 'TRUCK-TRACTOR', 'PICK UP TRUCK',
       'TRUCK (OTHER)', 'OFF ROAD - 4 WHEELS', 'TRUCK - DUMP',
       'OTHER EMERGENCY VEHICLE', 'POLICE VEHICLE', 'TOW TRUNK',
       'SCHOOL BUS', 'SCHOOL VAN', 'FIRE VEHICLE', 'OFF ROAD - 2 WHEELS',
       'STREET CAR', 'FARM TRACTOR', 'BUS (OTHER) (GO BUS, GRAY COACH)',
       'CONSTRUCTION EQUIPMENT', 'INTERCITY BUS', 'AMBULANCE',
       'TRUCK - TANK', 'OTHER SCHOOL VEHICLE/BUS', 'SNOW PLOW',
       'OFF ROAD - 3 WHEELS', 'TRUCK - CAR CARRIER',
       'MOTORIZED SNOW VEHICLE', 'MOTOR HOME', 'OTHER FARM VEHICLE',
       'OFF ROAD - OTHER', 'RAILWAY TRAIN'], dtype=object)

In [62]:
df2['traffic_control'].unique()

array(['NO CONTROL', 'TRAFFIC SIGNAL', 'STOP SIGN',
       'PEDESTRIAN CROSSOVER', 'YIELD SIGN', nan, 'TRAFFIC GATE',
       'POLICE CONTROL', 'SCHOOL GUARD', 'TRAFFIC CONTROLLER',
       'SCHOOL BUS', 'PXO - NO PED', 'STREETCAR (STOP FOR)'], dtype=object)

In [63]:
df2[df2['road_class']=='Other'].shape

(4974, 25)

---
##df3
---

In [66]:
# Create df3 from df2 for detecting most important features: Drop some unuseful columns
df3 = df2.copy()
df3.drop(['stname1', 'stname2', 'stname3', 'traffic_control', 'vehicle_no', 'vehicle_class'], axis=1, inplace=True)
df3.head()

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,impact_type,involved_class,involved_injury_class,acc_date,# involved,# injured,# fatalities,# KSI,hour,year,month,day
0,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0,8,2000,9,0
1,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0,8,2000,9,0
2,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0,8,2000,9,0
3,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0,8,2000,9,0
4,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,VEHICLE OWNER,NONE,2000-09-11,1,0,0,0,8,2000,9,0


In [67]:
for column in df3.columns:
    print('# nan in "{}": {:,}'.format(column, df3[column].isna().sum()))

# nan in "collision_no": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "road_class": 94,989
# nan in "visibility": 6,822
# nan in "light": 249
# nan in "road_surface_cond": 1,334
# nan in "impact_type": 279
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "acc_date": 0
# nan in "# involved": 0
# nan in "# injured": 0
# nan in "# fatalities": 0
# nan in "# KSI": 0
# nan in "hour": 0
# nan in "year": 0
# nan in "month": 0
# nan in "day": 0


In [68]:
# Drop nan rows from df3
df3.dropna(subset=['road_class', 'visibility', 'light', 'road_surface_cond', 'impact_type'], inplace=True )

for column in df3.columns:
    print('# nan in "{}": {:,}'.format(column, df3[column].isna().sum()))

# nan in "collision_no": 0
# nan in "longitude": 0
# nan in "latitude": 0
# nan in "road_class": 0
# nan in "visibility": 0
# nan in "light": 0
# nan in "road_surface_cond": 0
# nan in "impact_type": 0
# nan in "involved_class": 0
# nan in "involved_injury_class": 0
# nan in "acc_date": 0
# nan in "# involved": 0
# nan in "# injured": 0
# nan in "# fatalities": 0
# nan in "# KSI": 0
# nan in "hour": 0
# nan in "year": 0
# nan in "month": 0
# nan in "day": 0


In [69]:
df3.reset_index(drop=True, inplace=True)
df3

,collision_no,longitude,latitude,road_class,visibility,light,road_surface_cond,impact_type,involved_class,involved_injury_class,acc_date,# involved,# injured,# fatalities,# KSI,hour,year,month,day
0,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0,8,2000,9,0
1,1,-79.456317,43.727732,Minor Arterial,CLEAR,DAYLIGHT,DRY,TURNING MOVEMENT,DRIVER,NONE,2000-09-11,1,0,0,0,8,2000,9,0
2,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0,8,2000,9,0
3,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,PASSENGER,MINIMAL,2000-09-11,1,1,0,0,8,2000,9,0
4,2,-79.456271,43.727698,Minor Arterial,CLEAR,DAYLIGHT,DRY,OTHER,VEHICLE OWNER,NONE,2000-09-11,1,0,0,0,8,2000,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335519,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,TURNING MOVEMENT,DRIVER,NONE,2019-05-10,1,0,0,0,22,2019,5,4
2335520,1038396,-79.351940,43.677554,Major Arterial,CLEAR,"DARK, ARTIFICIAL",DRY,TURNING MOVEMENT,PASSENGER,MINOR,2019-05-10,1,1,0,0,22,2019,5,4
2335521,1038397,-79.314602,43.678378,Minor Arterial,RAIN,"DARK, ARTIFICIAL",WET,SMV UNATTENDED VEHICLE,DRIVER,NONE,2019-07-20,1,0,0,0,22,2019,7,5
2335522,1038398,-79.372333,43.651519,Major Arterial,CLEAR,DAYLIGHT,DRY,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,2019-10-08,1,1,0,0,9,2019,10,1


---
##df4
---

In [70]:
# Create df4 from df3
df4 = df3[['collision_no', 'year', 'month', 'day', 'hour', 'longitude', 'latitude', 
           'road_class', 'road_surface_cond', 'visibility', 'light', 'impact_type', 
           'involved_class', 'involved_injury_class', '# involved', '# injured', '# KSI', '# fatalities']]

df4.reset_index(drop=True, inplace=True)
df4

,collision_no,year,month,day,hour,longitude,latitude,road_class,road_surface_cond,visibility,light,impact_type,involved_class,involved_injury_class,# involved,# injured,# KSI,# fatalities
0,1,2000,9,0,8,-79.456317,43.727732,Minor Arterial,DRY,CLEAR,DAYLIGHT,TURNING MOVEMENT,DRIVER,NONE,1,0,0,0
1,1,2000,9,0,8,-79.456317,43.727732,Minor Arterial,DRY,CLEAR,DAYLIGHT,TURNING MOVEMENT,DRIVER,NONE,1,0,0,0
2,2,2000,9,0,8,-79.456271,43.727698,Minor Arterial,DRY,CLEAR,DAYLIGHT,OTHER,PASSENGER,MINIMAL,1,1,0,0
3,2,2000,9,0,8,-79.456271,43.727698,Minor Arterial,DRY,CLEAR,DAYLIGHT,OTHER,PASSENGER,MINIMAL,1,1,0,0
4,2,2000,9,0,8,-79.456271,43.727698,Minor Arterial,DRY,CLEAR,DAYLIGHT,OTHER,VEHICLE OWNER,NONE,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335519,1038396,2019,5,4,22,-79.351940,43.677554,Major Arterial,DRY,CLEAR,"DARK, ARTIFICIAL",TURNING MOVEMENT,DRIVER,NONE,1,0,0,0
2335520,1038396,2019,5,4,22,-79.351940,43.677554,Major Arterial,DRY,CLEAR,"DARK, ARTIFICIAL",TURNING MOVEMENT,PASSENGER,MINOR,1,1,0,0
2335521,1038397,2019,7,5,22,-79.314602,43.678378,Minor Arterial,WET,RAIN,"DARK, ARTIFICIAL",SMV UNATTENDED VEHICLE,DRIVER,NONE,1,0,0,0
2335522,1038398,2019,10,1,9,-79.372333,43.651519,Major Arterial,DRY,CLEAR,DAYLIGHT,PEDESTRIAN COLLISIONS,PEDESTRIAN,MINOR,1,1,0,0


In [71]:
df4.isna().sum()

collision_no             0
year                     0
month                    0
day                      0
hour                     0
longitude                0
latitude                 0
road_class               0
road_surface_cond        0
visibility               0
light                    0
impact_type              0
involved_class           0
involved_injury_class    0
# involved               0
# injured                0
# KSI                    0
# fatalities             0
dtype: int64

In [72]:
df4[df4['road_class']=='Pending'].shape

(1030, 18)

In [73]:
df4[df4['road_class']==' '].shape

(5, 18)

In [74]:
i = df4[df4['road_class'] == 'Pending'].index
df4.drop(i, inplace=True)

df4.reset_index(drop=True, inplace=True)

i = df4[df4['road_class'] == ' '].index
df4.drop(i, inplace=True)

df4.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [75]:
df_inKsi = df4[['involved_class', '# KSI']].groupby(['involved_class']).sum()
df_inKsi.reset_index(inplace=True)
df_inKsi.sort_values('# KSI', inplace=True, ascending=False)
df_inKsi

,involved_class,# KSI
14,PEDESTRIAN,3999
4,DRIVER,2681
13,PASSENGER,1359
1,CYCLIST,938
9,MOTORCYCLE DRIVER,644
18,TRUCK DRIVER,31
7,MOPED DRIVER,28
10,MOTORCYCLE PASSENGER,23
21,WHEELCHAIR,17
11,OTHER,7


In [76]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Bar(
    x = df_inKsi['involved_class'], 
    y = df_inKsi['# KSI'],
    # text = feature_imp,
    # textposition = 'outside',
    # texttemplate = '%{text:.2f}'+'%'
    )])
fig.update_layout(title = '',
                  xaxis = dict(
                      title = 'Involved People'), 
                  yaxis = dict(
                      title = '# KSI'
                  ))
fig.show()

In [0]:
df4.to_csv('df4.csv')